# Shearwater Bay - Development Plan

In [1]:
# to create data tables
import geopandas as gpd
import pandas as pd

# to transform UTM to WGS84
from shapely.geometry import Polygon
from pyproj import Proj, transform

# to create the interactive map
import folium
from folium import Popup
from folium.plugins import MarkerCluster
import json

# to parse the KML files and plot the polygons
from fastkml import kml
from shapely.geometry import Polygon, Point, LineString
from xml.dom.minidom import parseString

# Add widgets
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date

# ignore warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pyproj")
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Insert license areas

In [2]:
# Function to Extract Coordinates from KML
def extract_coordinates_from_kml(kml_path):
    """Extract polygon coordinates from a KML file."""
    with open(kml_path, 'r') as f:
        doc = f.read()

    # Parse the KML content
    kml = parseString(doc)
    coordinates = []

    # Find all Polygon coordinates in the KML
    for placemark in kml.getElementsByTagName('Placemark'):
        for polygon in placemark.getElementsByTagName('Polygon'):
            for outer_boundary in polygon.getElementsByTagName('outerBoundaryIs'):
                for linear_ring in outer_boundary.getElementsByTagName('LinearRing'):
                    for coord in linear_ring.getElementsByTagName('coordinates'):
                        coords_text = coord.firstChild.nodeValue.strip()
                        coords = coords_text.split(" ")
                        coords = [(float(c.split(",")[1]), float(c.split(",")[0])) for c in coords]
                        coordinates.append(coords)
    return coordinates

In [3]:
# List of KML Files
kml_files = [
    'Development Plan - Input/Plot A - 100 ha.kml',
    'Development Plan - Input/Plot B - 100 ha.kml',
    'Development Plan - Input/Plot C - 115 ha.kml',
    'Development Plan - Input/Plot D - 105 ha.kml',
    'Development Plan - Input/Plot E - 100 ha.kml',
    'Development Plan - Input/Plot F - 100 ha.kml']

### Insert data table

In [5]:
xl = pd.ExcelFile('SWB - Installation Phasing - Shallow_2.xlsx')
# xl = pd.ExcelFile('SWB - Installation Phasing.xlsx')
df_UTM = xl.parse(xl.sheet_names[0])
df_UTM = df_UTM.dropna(subset=['Easting', 'Northing'])
# df_UTM['Date'] = pd.to_datetime(df_UTM['Date']).dt.date
df_UTM['Label'] = df_UTM['Label'].apply(lambda x: f"{int(x):03}" if not pd.isna(x) else x)

In [6]:
df_UTM.iloc[10:15].head()

,#,Tag,Country,Site,Plot,Label,Type,Design,Fill,Outline,...,Easting,Northing,NW-E,NW-N,NE-E,NE-N,SE-E,SE-N,SW-E,SW-N
10,11,NA-SB-K,NaN,SB,K,NaN,-,Shallow,Grey,Grey,...,510300.0,7053570.0,510279.0,7053615.0,510321.0,7053615.0,510321.0,7053525.0,510279.0,7053525.0
11,12,NA-SB-L-01,NaN,SB,L,001,BB,Shallow,Green,Green,...,510170.0,7053500.0,510120.0,7053550.0,510220.0,7053550.0,510220.0,7053450.0,510120.0,7053450.0
12,13,NA-SB-L-02,NaN,SB,L,002,BB,Shallow,Green,Green,...,510070.0,7053500.0,510020.0,7053550.0,510120.0,7053550.0,510120.0,7053450.0,510020.0,7053450.0
13,14,NA-SB-L-03,NaN,SB,L,003,BB,Shallow,Green,Green,...,509970.0,7053500.0,509920.0,7053550.0,510020.0,7053550.0,510020.0,7053450.0,509920.0,7053450.0
14,15,NA-SB-L-04,NaN,SB,L,004,BB,Shallow,Green,Green,...,509870.0,7053500.0,509820.0,7053550.0,509920.0,7053550.0,509920.0,7053450.0,509820.0,7053450.0


In [ ]:
# Define the Proj objects for the transformations
utm33 = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
wgs84 = Proj(proj='latlong', datum='WGS84')

In [ ]:
# Initialize df_latlon as a copy of df_UTM so that original data is not modified
df_latlon = df_UTM.copy()

coordinate_columns = [('Easting', 'Northing'), 
                      ('NW-E', 'NW-N'), 
                      ('NE-E', 'NE-N'), 
                      ('SE-E', 'SE-N'), 
                      ('SW-E', 'SW-N')]

def utm_to_latlon(row, easting_col, northing_col):
    # convert UTM to lat/lon
    northing = 10000000 - row[northing_col]  # adjust for southern hemisphere
    lon, lat = transform(utm33, wgs84, row[easting_col], northing)
    return pd.Series({f'{easting_col}_lat': -lat, f'{easting_col}_lon': lon})

# Convert coordinates for each pair of columns and rename
rename_dict = {'Easting': 'lat', 'Northing': 'lon'}
for easting_col, northing_col in coordinate_columns:
    df_latlon[[f'{easting_col}_lat', f'{easting_col}_lon']] = df_latlon.apply(lambda row: utm_to_latlon(row, easting_col, northing_col), axis=1)
    base_name = easting_col if easting_col == "Easting" else easting_col.split('-')[0].strip()
    rename_dict[f'{easting_col}_lat'] = f"{base_name}_lat"
    rename_dict[f'{easting_col}_lon'] = f"{base_name}_lon"

# Rename the columns in df_latlon
df_latlon.rename(columns=rename_dict, inplace=True)

# Display the first few rows of the df_latlon DataFrame
# df_latlon.head()

## Insert license areas

In [ ]:
def transform_filename(filename):
    # Replace 'Plot' with 'License Area' and remove the '.kml' extension
    return filename.replace("Plot", "License Area").replace(".kml", "")

In [ ]:
# Extract Coordinates from All KML Files
all_coordinates = {file.split("/")[-1]: extract_coordinates_from_kml(file) for file in kml_files}

# Create Folium Map
m_license_area = folium.Map(tiles='CartoDB Positron',location=[df_latlon['Easting_lat'].mean(), df_latlon['Easting_lon'].mean()], zoom_start=14)

# Add Polygons to the Map
for filename, coords in all_coordinates.items():
    for polygon in coords:
        # Transform the filename to the desired popup format
        popup_content = transform_filename(filename)
        tooltip_content = transform_filename(filename)
#         popup = Popup(popup_content, max_width=300)  # Adjust max_width as needed
        
        folium.Polygon(
            locations=polygon, 
            fill=True,
            tooltip=tooltip_content,
            color="#3388ff",
            weight=1,
            fill_color="#3388ff",
            fill_opacity=0.2
        ).add_to(m_license_area)

m_license_area.save("Development Plan - Output/license_area_map.html")
# m_license_area

In [ ]:
# Sort df_latlon by "Date" in ascending order
df_latlon_sorted = df_latlon.sort_values(by="Date")

# Extract unique combinations for each pair of columns, retaining the earliest "To be installed by" date
unique_NW = df_latlon_sorted[['NW_lat', 'NW_lon', 'Marks', 'Plot', 'Date']].drop_duplicates(subset=['NW_lat', 'NW_lon']).reset_index(drop=True)
unique_NE = df_latlon_sorted[['NE_lat', 'NE_lon', 'Marks', 'Plot', 'Date']].drop_duplicates(subset=['NE_lat', 'NE_lon']).reset_index(drop=True)
unique_SE = df_latlon_sorted[['SE_lat', 'SE_lon', 'Marks', 'Plot', 'Date']].drop_duplicates(subset=['SE_lat', 'SE_lon']).reset_index(drop=True)
unique_SW = df_latlon_sorted[['SW_lat', 'SW_lon', 'Marks', 'Plot', 'Date']].drop_duplicates(subset=['SW_lat', 'SW_lon']).reset_index(drop=True)

# Concatenate the unique combinations into a new DataFrame called df_anchor
df_anchor = pd.concat([
    unique_NW.rename(columns={'NW_lat': 'lat', 'NW_lon': 'lon'}),
    unique_NE.rename(columns={'NE_lat': 'lat', 'NE_lon': 'lon'}),
    unique_SE.rename(columns={'SE_lat': 'lat', 'SE_lon': 'lon'}),
    unique_SW.rename(columns={'SW_lat': 'lat', 'SW_lon': 'lon'})
]).reset_index(drop=True)


df_anchor = df_anchor.sort_values(by="Date").reset_index(drop=True)
df_anchor_unique = df_anchor.drop_duplicates(subset=['lat', 'lon'], keep='first')
df_anchor_unique = df_anchor_unique[df_anchor_unique["Marks"] != "-"]
df_anchor_unique.head(40)

In [ ]:
# Generating anchor tags
def generate_anchor_tags(group):
    plot_name = group['Plot'].iloc[0]
    count = len(group)
    anchor_tags = ["NA-SB-{}-{:02d}".format(plot_name, i) for i in range(1, count + 1)]
    return pd.Series(anchor_tags, index=group.index)

df_anchor_tag = df_anchor_unique
df_anchor_tag['Tag'] = df_anchor_unique.groupby('Plot').apply(generate_anchor_tags).reset_index(level=0, drop=True)
df_anchor_tag = df_anchor_tag.reset_index(drop=True)

df_anchor_tag.head(35)

In [ ]:
# Initialize a new Folium map centered around the mean of the anchor points
m_anchor = folium.Map(location=[df_anchor_tag['lat'].mean(), df_anchor_tag['lon'].mean()], zoom_start=15, tiles='CartoDB Positron')

# Plot each unique anchor point on the map with a popup displaying the "To be installed by" date
for idx, row in df_anchor_tag.iterrows():
    popup_content = f'<div style="text-align: center;">{row["Tag"]}<br>Install by {row["Date"]}</div>'
    popup = Popup(popup_content, max_width=300)

    # Create tooltip content with centered text
    tooltip_content = f'<div style="text-align: center;">{row["Tag"]}<br>lat {row["lat"]:.5f}; lon {row["lon"]:.5f}<br>Install by {row["Date"]}</div>'
    
    folium.CircleMarker(
        location=(row['lat'], row['lon']),
        radius=2,
        color=row['Marks'],  # Set the color using the "Marks" column
        fill=True,
        fill_color=row['Marks'],  # Set the fill color using the "Marks" column
        fill_opacity=1,
        popup=popup,
        tooltip=tooltip_content
    ).add_to(m_anchor)
    
m_anchor.save("Development Plan - Output/screw_anchor_map.html")
m_anchor

In [ ]:
# Adjust the column sequence
desired_columns = ['Tag', 'lat', 'lon']
df_anchor_tag_adjusted = df_anchor_tag[desired_columns]

# Export the df_anchor_tag DataFrame to an Excel (.xlsx) file
output_file_path = "Development Plan - Output/df_anchor_tag.xlsx"
df_anchor_tag_adjusted.to_excel(output_file_path, index=False)
df_anchor_tag_adjusted.head()

In [ ]:
# Extract Coordinates from All KML Files
all_coordinates = {file.split("/")[-1]: extract_coordinates_from_kml(file) for file in kml_files}

# Create Folium Map
m_polygons = folium.Map(tiles='CartoDB Positron',location=[df_latlon['Easting_lat'].mean(), df_latlon['Easting_lon'].mean()], zoom_start=15)

# Add Polygons to the Map
for filename, coords in all_coordinates.items():
    for polygon in coords:

        popup_content = transform_filename(filename)
        popup = Popup(popup_content, max_width=300)  

        folium.Polygon(
            locations=polygon, 
            popup=popup,
            fill=True,
            color="#3388ff",
            weight=1,
            fill_color="#3388ff",
            fill_opacity=0.2
        ).add_to(m_polygons)

# Add a color-coded polygon for each point
for idx, row in df_latlon.iterrows():
    
    # Create tooltip content with centered text
    tooltip_content = f'<div style="text-align: center;">{row["Tag"]}</div>'
    popup_content = f'<div style="text-align: center;">{row["Tag"]}<br>Install by {row["Date"]}</div>'
    popup = Popup(popup_content, max_width=300)  
    
    polygon_points = [
        [row['NW_lat'], row['NW_lon']],
        [row['NE_lat'], row['NE_lon']],
        [row['SE_lat'], row['SE_lon']],
        [row['SW_lat'], row['SW_lon']]
    ]
    
    folium.Polygon(polygon_points, 
                   color=row['Fill'],
                   weight=1.5,
                   fill_color=row['Fill'], 
                   fill_opacity=0.5,
                   tooltip=tooltip_content,
                   popup=popup
                  ).add_to(m_polygons)

# Iterate over the df_anchor_tag DataFrame to add color-coded dot for each anchor
for idx, row in df_anchor_tag.iterrows():
    
    marker_color = row['Marks']  # Directly get color from 'Marks' column

    tooltip_content = row["Tag"]
    popup_content = tooltip_content
    popup = Popup(popup_content, max_width=300)  

    
    if marker_color != "-":  # Only add the marker if the color is not "Blank"
        folium.CircleMarker([row['lat'], row['lon']], 
                            radius=2, 
                            tooltip=tooltip_content,  # Use "Anchor Tag" for tooltip
                            popup=popup,  # Use "Anchor Tag" for tooltip
                            color=marker_color,  # Use the color from the 'Marks' column
                            fill=True, 
                            fill_color=marker_color, 
                            fill_opacity=1,
                            ).add_to(m_polygons)

m_polygons.save("Development Plan - Output/full_map.html")
m_polygons

In [ ]:
# Create the Date Range Picker
start_date_widget = widgets.DatePicker(description='Start Date', value=date(2023, 9, 1))
end_date_widget = widgets.DatePicker(description='End Date', value=date(2023, 12, 24))

# Display the widgets
display(start_date_widget)
display(end_date_widget)

button = widgets.Button(description="Update Map")

def on_button_click(b):
    clear_output(wait=True)
    
    # Filtering data based on widget values
    filtered_df = df_anchor_tag[
        (df_anchor_tag['Date'] >= pd.Timestamp(start_date_widget.value)) & 
        (df_anchor_tag['Date'] <= pd.Timestamp(end_date_widget.value))
    ]
    
    # Code to plot the filtered data on the folium map
    m_anchor_window = folium.Map(location=[filtered_df['lat'].mean(), filtered_df['lon'].mean()], zoom_start=15, tiles='CartoDB Positron')
        
    # Iterate over the df_anchor_tag DataFrame to add color-coded dot for each anchor
    for idx, row in filtered_df.iterrows():

        marker_color = row['Marks']  # Directly get color from 'Marks' column

        tooltip_content = f'<div style="text-align: center;">{row["Tag"]}<br>lat {row["lat"]:.5f}; lon {row["lon"]:.5f}<br>Install by {row["Date"]}</div>'
        popup_content = f'<div style="text-align: center;">{row["Tag"]}<br>Install by {row["Date"]}</div>'
        popup = Popup(popup_content, max_width=300)  

        if marker_color != "-":  # Only add the marker if the color is not "Blank"
            folium.CircleMarker([row['lat'], row['lon']], 
                                radius=2, 
                                tooltip=tooltip_content,  # Use "Anchor Tag" for tooltip
                                popup=popup,  # Use "Anchor Tag" for tooltip
                                color=marker_color,  # Use the color from the 'Marks' column
                                fill=True, 
                                fill_color=marker_color, 
                                fill_opacity=1,
                                ).add_to(m_anchor_window)
    
    display(m_anchor_window)
          
    m_anchor_window.save("Development Plan - Output/screw_anchor_map_window.html")
    m_anchor_window
    
    # Redisplay everything
    display(start_date_widget)
    display(end_date_widget)
    display(button)
    
button.on_click(on_button_click)
display(button)

In [ ]:
# Create the Date Range Picker with default values
start_date_widget = widgets.DatePicker(description='Start Date', value=date(2023, 9, 1))
end_date_widget = widgets.DatePicker(description='End Date', value=date(2023, 12, 24))

# Display the widgets
display(start_date_widget)
display(end_date_widget)

button = widgets.Button(description="Update Map")

def on_button_click(b):
    clear_output(wait=True)
    
    # Convert the 'Date' columns to datetime format if they're not already
    df_latlon['Date'] = pd.to_datetime(df_latlon['Date'])
    df_anchor_tag['Date'] = pd.to_datetime(df_anchor_tag['Date'])

    # Filter based on the adjusted date comparisons
    start_date = pd.Timestamp(start_date_widget.value).date()
    end_date = pd.Timestamp(end_date_widget.value).date()
    
    filtered_df_polygon = df_latlon[
        (df_latlon['Date'].dt.date >= start_date) & 
        (df_latlon['Date'].dt.date <= end_date)
    ]
    
    filtered_df_anchor = df_anchor_tag[
        (df_anchor_tag['Date'].dt.date >= start_date) & 
        (df_anchor_tag['Date'].dt.date <= end_date)
    ]
   
    # Code to plot the filtered data on the folium map
    m_window = folium.Map(location=[filtered_df_anchor['lat'].mean(), filtered_df_anchor['lon'].mean()], zoom_start=15, tiles='CartoDB Positron')
    
    # Add Polygons to the Map
    for filename, coords in all_coordinates.items():
        for polygon in coords:

            popup_content = transform_filename(filename)
            popup = Popup(popup_content, max_width=300)  

            folium.Polygon(
                locations=polygon, 
                popup=popup,
                fill=True,
                color="#3388ff",
                weight=1,
                fill_color="#3388ff",
                fill_opacity=0.2
            ).add_to(m_window)
            
    # Add a color-coded polygon for each point
    for idx, row in filtered_df_polygon.iterrows():

        # Create tooltip content with centered text
        tooltip_content = f'<div style="text-align: center;">{row["Tag"]}</div>'

        polygon_points = [
            [row['NW_lat'], row['NW_lon']],
            [row['NE_lat'], row['NE_lon']],
            [row['SE_lat'], row['SE_lon']],
            [row['SW_lat'], row['SW_lon']]
        ]

        folium.Polygon(polygon_points, 
                       color=row['Fill'], 
                       fill_color=row['Fill'], 
                       fill_opacity=0.5,
                       tooltip=tooltip_content
                      ).add_to(m_window)
            
    # Add a color-coded polygon for each point
    for idx, row in filtered_df_polygon.iterrows():

        # Create tooltip content with centered text
        tooltip_content = f'<div style="text-align: center;">{row["Tag"]}</div>'
        popup_content = f'<div style="text-align: center;">{row["Tag"]}<br>Install by {row["Date"].date()}</div>'
        popup = Popup(popup_content, max_width=300)  

        polygon_points = [
            [row['NW_lat'], row['NW_lon']],
            [row['NE_lat'], row['NE_lon']],
            [row['SE_lat'], row['SE_lon']],
            [row['SW_lat'], row['SW_lon']]
        ]

        folium.Polygon(polygon_points, 
                       color=row['Fill'],
                       weight=1.5,
                       fill_color=row['Fill'], 
                       fill_opacity=0.3,
                       tooltip=tooltip_content,
                       popup=popup
                      ).add_to(m_window)
    
    # Iterate over the df_anchor_tag DataFrame to add color-coded dot for each anchor
    for idx, row in filtered_df_anchor.iterrows():

        marker_color = row['Marks']  # Directly get color from 'Marks' column

        tooltip_content = f'<div style="text-align: center;">Helical screw anchor:<br>{row["Tag"]}<br>lat {row["lat"]:.5f}; lon {row["lon"]:.5f}<br>Install by {row["Date"].date()}</div>'
        popup_content = f'<div style="text-align: center;">{row["Tag"]}<br>Install by {row["Date"].date()}</div>'
        popup = Popup(popup_content, max_width=300)  

        if marker_color != "-":  # Only add the marker if the color is not "Blank"
            folium.CircleMarker([row['lat'], row['lon']], 
                                radius=2, 
                                tooltip=tooltip_content,  # Use "Anchor Tag" for tooltip
                                popup=popup,  # Use "Anchor Tag" for tooltip
                                color=marker_color,  # Use the color from the 'Marks' column
                                fill=True, 
                                fill_color=marker_color, 
                                fill_opacity=1,
                                ).add_to(m_window)
    
    display(m_window)
          
    m_window.save("Development Plan - Output/full_map_window.html")
    m_window
    
    # Redisplay everything
    display(start_date_widget)
    display(end_date_widget)
    display(button)
        
button.on_click(on_button_click)
display(button)

In [ ]:
# Load the newly uploaded Excel file to inspect its contents
df_anchor_L = pd.read_excel("Development Plan - Output/df_anchor_tag_L.xlsx", sheet_name=0)

# Initialize a map centered around the first coordinate
m = folium.Map(location=[df_anchor_L['lat'][0], df_new_2['lon'][0]], zoom_start=10)

# Add markers for each coordinate
for index, row in df_anchor_L.iterrows():
    folium.Marker([row['lat'], row['lon']], tooltip=row['Tag']).add_to(m)

# Display the map
m

In [ ]:
from pyproj import Proj, transform

# Set up the projection for UTM Zone 33S
proj_utm = Proj(proj='utm', zone=33, south=True, ellps='WGS84', datum='WGS84')
proj_latlon = Proj(proj='latlong', ellps='WGS84', datum='WGS84')

# Convert lat and lon to UTM coordinates
def latlon_to_utm_alternative(lat, lon):
    easting, northing = transform(proj_latlon, proj_utm, lon, lat)
    return easting, northing

df_anchor_L['easting'], df_anchor_L['northing'] = zip(*df_anchor_L.apply(lambda x: latlon_to_utm_alternative(x['lat'], x['lon']), axis=1))

# Display the updated dataframe with UTM coordinates
df_anchor_L.head()

In [ ]:
# Sort the dataframe
df_sorted = df_anchor_L.sort_values(by=['northing', 'easting'], ascending=[True, True])

# Assign the "Anchor Tag"
df_sorted['Anchor Tag'] = ['A-' + str(i+1) for i in range(len(df_sorted))]

# Display the updated dataframe
df_sorted.head()

In [ ]:
# Initialize a folium map centered on the mean latitude and longitude of the dataset
m = folium.Map(location=[df_sorted['lat'].mean(), df_sorted['lon'].mean()], zoom_start=15)

# Add markers for each anchor with the "Anchor Tag" and UTM coordinates
for idx, row in df_sorted.iterrows():
    folium.Marker(
        location=(row['lat'], row['lon']),
        popup=f"Anchor Tag: {row['Anchor Tag']}<br>Easting: {row['easting']}<br>Northing: {row['northing']}",
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

m